In [ ]:
import os
import sys
import numpy as np

import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

from sklearn.manifold import TSNE

import matplotlib.pyplot as plt

sys.path.append("/home/self_supervised_learning_gr/self_supervised_learning/dev/ProjetCassiopee")
from src.setup import setup_python, setup_pytorch
from src.dataset import RGBDObjectDataset, RGBDObjectDataset_Contrast
from src.models.combined_model import CombinedModel, train, test

In [ ]:
setup_python()

In [ ]:
DEVICE = setup_pytorch()

In [ ]:
BATCH_SIZE = 10 # Batch size
INPUT_SIZE = (128,128)
TRANSFORMATION = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(size=INPUT_SIZE)])
NB_TEST_SAMPLES = 100

test_dataset = RGBDObjectDataset_Contrast(path="data/RGB-D_Object/rgbd-dataset",
                                          mode="test",
                                          transformation=TRANSFORMATION,
                                          nb_samples=NB_TEST_SAMPLES)

test_data_loader = DataLoader(test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

In [ ]:
model = CombinedModel().to(DEVICE)

In [ ]:
# Run inference
encoded_features = []
labels = []
with torch.no_grad():
    for i, batch in enumerate(test_data_loader):
        
        # Load and prepare batch
        p_data_1, p_data_2, n_data = batch
        p_rgb_1, p_depth_1, p_mask_1, p_loc_x_1, p_loc_y_1, p_label_1 = p_data_1
        p_rgb_1 = p_rgb_1.to(DEVICE)
        # p_depth_1 = p_depth_1.to(DEVICE)
        # p_mask_1 = p_mask_1.to(DEVICE)
        # p_loc_x_1 = p_loc_x_1.to(DEVICE)
        # p_loc_y_1 = p_loc_y_1.to(DEVICE)
        p_label_1 = p_label_1.to(DEVICE)
        # p_rgb_2, p_depth_2, p_mask_2, p_loc_x_2, p_loc_y_2, p_label_2 = p_data_2
        # p_rgb_2 = p_rgb_2.to(DEVICE)
        # p_depth_2 = p_depth_2.to(DEVICE)
        # p_mask_2 = p_mask_2.to(DEVICE)
        # p_loc_x_2 = p_loc_x_2.to(DEVICE)
        # p_loc_y_2 = p_loc_y_2.to(DEVICE)
        # p_label_2 = p_label_2.to(DEVICE)
        # n_rgb, n_depth, n_mask, n_loc_x, n_loc_y, n_label = n_data
        # n_rgb = n_rgb.to(DEVICE)
        # n_depth = n_depth.to(DEVICE)
        # n_mask = n_mask.to(DEVICE)
        # n_loc_x = n_loc_x.to(DEVICE)
        # n_loc_y = n_loc_y.to(DEVICE)
        # n_label = n_label.to(DEVICE)
        
        # Make predictions for batch
        encoded_x, predicted_label = model(p_rgb_1)

        # Save encoded features and labels
        encoded_features.append(encoded_x)
        labels.append(p_label_1)

In [ ]:
print(len(encoded_features))
print(encoded_features[0].shape)
print(encoded_features[0])

In [ ]:
encoded_features_arr = torch.empty(size=(NB_TEST_SAMPLES, 4096))
print(encoded_features_arr.shape)
print(encoded_features_arr)
for i, batch in enumerate(encoded_features):
    encoded_features_arr[i * BATCH_SIZE:(i + 1) * BATCH_SIZE,:] = batch
print(encoded_features_arr.shape)
print(encoded_features_arr)

In [ ]:
print(len(labels))
print(labels[0].shape)
print(labels[0])

In [ ]:
labels_arr = torch.empty(size=(NB_TEST_SAMPLES,))
print(labels_arr.shape)
print(labels_arr)
for i, batch in enumerate(labels):
    labels_arr[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] = batch
print(labels_arr.shape)
print(labels_arr)

In [ ]:
# Apply TSNE
tsne = TSNE(n_components=2,
            perplexity=30,
            n_iter=1000,
            init="pca",
            random_state=42)

In [ ]:
tsne_results = tsne.fit_transform(encoded_features_arr)

In [ ]:
print(type(tsne_results))
print(tsne_results.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(tsne_results[:,0], tsne_results[:,1], c=labels_arr, s=50, alpha=0.8)